In [ ]:
using CSV
using Plots
using DataFrames
using StatsPlots, KernelDensity
using PyCall

theme(:dracula)

In [ ]:
surp = pyimport("surp")

In [ ]:
subgiants = surp.src.analysis.apogee_analysis.subgiants

function pd_to_df(df_pd)
    df= DataFrame()
    for col in df_pd.columns
        df[!, col] = getproperty(df_pd, col).values
    end
    df
end

sg = pd_to_df(subgiants);

In [ ]:
mg_h_err(mg_h) = max(0.015,-0.03*mg_h + 0.025) * randn()

c_mg_err(mg_h) = max(0.02, -0.11*mg_h + 0.04) * randn()

mg_fe_err(mg_h) = max(0.01, -0.0225*mg_h+0.02) * randn()

In [ ]:
df = CSV.read("../../results/C11_f0.2_eta1_beta0.001_lateburst1.5.csv", DataFrame);
mg_h = df[:, "MG_H"]

df[:, "MG_H"] .+= mg_h_err.(mg_h)
df[:, "MG_FE"] .+= mg_fe_err.(mg_h)
df[:, "C_MG"] .+= c_mg_err.(mg_h);

In [ ]:
theme(:dracula, background_color="black")

In [ ]:
dz = 0.05

anim = @animate for mg_h in -0.2:0.01:0.5
    
    filt = @. mg_h <= sg[:,"MG_H"] < mg_h + dz
    filt .&= (sg[:,"high_alpha"] .!= 1)
    filt .&= isfinite.(sg[:,"C_MG"])
    dens = kde((sg[filt, "MG_FE"], sg[filt, "C_MG"]))
    plot(dens, clim=(0, 200), legend=false,
        background_color="black",
        background_color_subplot="black",
        background_color_inside="black"
        
    )
    
    filt = @. mg_h-dz < df[:,"MG_H"] < mg_h + dz
    if sum(filt) > 1
        scatter!(df[filt, "MG_FE"], df[filt, "C_MG"], ms=1, alpha=0.2,
            background_color_subplot="black",
            grid=:none
        )
    end

    
    
    xlims!(-0.1, 0.3)
    ylims!(-0.4, 0.1)
    xlabel!("[α/Fe]")
    ylabel!("[C/α]")
end

gif(anim, fps=10)